In [54]:
import pandas as pd
import numpy as np
import csv
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import pickle

In [55]:
# import holdout sales data from King Country, WA
df = pd.read_csv('kc_house_data_test_features.csv', index_col = 0)
pd.set_option('display.max_columns', None) # display all columns

# data cleanup and add features
df.isna().sum(); # there are no blank values

# add features
# add yard size
(df.sqft_living == df.sqft_above + df.sqft_basement).value_counts(); # check: all True
df['sqft_yard'] = df.sqft_lot - df.sqft_above/df.floors # calculate yard size
# note: cannot calculate yard size of neighbors, missing 'sqft_above15' and 'floors15'
df['sqrt_yard'] = np.sqrt(np.abs(df.sqft_yard))
# log base 10 of price

# bathrooms
(df.bathrooms[df.bathrooms<0.5]).value_counts(); # found 8 units without even a half bath
df['no_bath'] = np.where(df.bathrooms == 0, 1, 0) # create new variable for has no bathroom
# df.bathrooms = np.where(df.bathrooms == 0, np.nan, df.bathrooms) # replace with np.nan

# basements
(df.sqft_basement[df.sqft_basement==0]).value_counts(); # found many houses without basements
df['has_base'] = np.where(df.sqft_basement == 0, 0, 1) # create new variable for has basement
# df.sqft_basement = np.where(df.sqft_basement == 0, np.nan, df.sqft_basement) # replace with np.nan

# year built vs year renovated
# set year = yr_built, or yr_renovated, whichever is more recent
df['year'] = np.where(df.yr_renovated==0,df.yr_built,df.yr_renovated);
df['reno'] = np.where(df.yr_renovated!=0, 1, 0) # create new variable for has been renovated

# distance from middle of year range
# define the pivot year as 1956 and create new feature
mid_year = 1956
df['year_mid'] = np.abs(df['year'] - mid_year)

# import dictionary of zips ranked from cheapest median home prices to most expensive
with open('zip_ranks_cheapest_first.csv', newline='', encoding='utf-8-sig') as csv_file:
    my_reader = csv.reader(csv_file)
    zip_dict = {int(row[0]):int(row[1]) for row in my_reader}
df['zip_rank'] = df.zipcode.map(zip_dict)

# find month of sale
df['month'] = pd.DatetimeIndex(df.date).month

# add a polynomial variable
df['grade_2'] = df.grade^2 # grade has a strong correlation

# create interaction variable
df['living_bath'] = df.sqft_living*df.no_bath # having no bathroom makes it hard to live

In [56]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'view', 'month',
            'grade', 'grade_2', 'condition', 'zip_rank', 'sqft_living15', 'year_mid', 'year',
            'sqft_lot15', 'has_base', 'reno', 'living_bath', 'sqft_basement', 'sqrt_yard',
            'sqft_yard', 'zipcode'
           ]
df_test = df[features]
df_dummy = pd.get_dummies(df.zipcode)
df_test = pd.concat([df_test.drop('zipcode', axis = 1),df_dummy], axis = 1)

In [57]:
df_test

,bedrooms,bathrooms,sqft_living,floors,waterfront,view,month,grade,grade_2,condition,zip_rank,sqft_living15,year_mid,year,sqft_lot15,has_base,reno,living_bath,sqft_basement,sqrt_yard,sqft_yard,98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,4,2.50,2270,1.0,0,0,8,8,10,3,38,2020,11,1967,10918,1,0,0,730,99.799800,9960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2.50,2270,1.0,0,0,2,8,10,3,38,2020,11,1967,10918,1,0,0,730,99.799800,9960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2.50,1470,2.0,0,0,11,8,10,3,47,1470,49,2005,1576,1,0,0,310,34.626579,1199.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1.75,1280,1.0,0,0,12,8,10,3,69,1160,20,1976,10565,1,0,0,250,123.166554,15170.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2.75,2830,2.0,0,0,1,8,10,3,35,2830,49,2005,7916,0,0,0,0,81.920693,6711.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3,2.50,1530,3.0,0,0,5,8,10,3,61,1530,53,2009,1509,0,0,0,0,24.919872,621.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4319,4,2.50,2310,2.0,0,0,2,8,10,3,24,1830,58,2014,7200,0,0,0,0,68.249542,4658.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4320,2,0.75,1020,2.0,0,0,6,7,5,3,50,1020,53,2009,2007,0,0,0,0,28.982753,840.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4321,3,2.50,1600,2.0,0,0,1,8,10,3,51,1410,48,2004,1287,0,0,0,0,39.849718,1588.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [65]:
# load model
infile = open("model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [78]:
# run predictions and exponentiate to arrive at $ value
price_pred = 10**model.predict(df_test)
price_pd = pd.DataFrame(price_pred, )
price_pd
price_pd.to_csv('housing_preds_wei_alex_xin.csv')

In [83]:
with open('housing_preds_wei_alex_xin.csv', mode = 'w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter = '\n')
    csv_writer.writerow(['price'])
    csv_writer.writerow(price_pred)